In [8]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from shapely.wkt import loads
from city_road_network.utils.utils import get_data_subdir, get_html_subdir
import time
import pickle
from city_road_network.utils.map import draw_trips_map
from city_road_network.utils.io import read_graph
from city_road_network.algo.smarter_sim import run_smarter_simulation
from city_road_network.config import default_crs

In [9]:
city_name = "moscow"
data_dir = get_data_subdir(city_name)
html_dir = get_html_subdir(city_name)

In [10]:
G = read_graph(
    os.path.join(data_dir, "nodelist_upd.csv"),
    os.path.join(data_dir, "edgelist_upd.csv"),
)

/home/nikita/Projects/city_road_network/city_road_network/utils/io.py:17: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  edgelist = pd.read_csv(


In [11]:
trip_mat = np.load(os.path.join(data_dir, "trip_mat.npy")) // 2
n = trip_mat.shape[0]

In [12]:
zones_df = pd.read_csv(os.path.join(data_dir, "zones_upd.csv"), index_col=0)
zones_df["geometry"] = zones_df["geometry"].apply(loads)
zones_gdf = gpd.GeoDataFrame(zones_df, crs=default_crs)

In [13]:
all_paths, new_graph = run_smarter_simulation(G, trip_mat, "flow_time (s)")
for i in range(len(all_paths)):
    for j in range(len(all_paths)):
        assert len(all_paths[i][j]) == trip_mat[i, j]

[2023-05-07 20:46:14,189][INFO] Processed chunk...
[2023-05-07 20:52:02,044][INFO] Processed chunk...
[2023-05-07 20:56:34,644][INFO] Processed chunk...
[2023-05-07 21:00:30,448][INFO] Processed chunk...
[2023-05-07 21:04:11,897][INFO] Processed chunk...
[2023-05-07 21:07:36,311][INFO] Processed chunk...
[2023-05-07 21:10:48,560][INFO] Processed chunk...
[2023-05-07 21:14:06,292][INFO] Processed chunk...
[2023-05-07 21:17:13,947][INFO] Processed chunk...
[2023-05-07 21:20:15,322][INFO] Processed chunk...
[2023-05-07 21:23:04,737][INFO] Processed chunk...
[2023-05-07 21:25:39,636][INFO] Processed chunk...
[2023-05-07 21:28:07,129][INFO] Processed chunk...
[2023-05-07 21:30:23,206][INFO] Processed chunk...
[2023-05-07 21:32:29,368][INFO] Processed chunk...
[2023-05-07 21:34:27,598][INFO] Processed chunk...
[2023-05-07 21:36:18,367][INFO] Processed chunk...
[2023-05-07 21:38:01,784][INFO] Processed chunk...
[2023-05-07 21:39:25,676][INFO] Processed chunk...
[2023-05-07 21:40:34,649][INFO]

In [14]:
ts = int(time.time())
map = draw_trips_map(new_graph)
filename = f"smarter_paths_by_time_{ts}"
map.save(os.path.join(html_dir, f"{filename}.html"))
with open(os.path.join(data_dir, f"{filename}.pkl"), "wb") as f:
    pickle.dump(all_paths, f)